<a href="https://colab.research.google.com/github/HCI-hope/Models/blob/main/Chebyshev_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.signal import cheby1, filtfilt
from google.colab import drive
from pathlib import Path

# MOUNT GOOGLE DRIVE (Skip if already mounted)
# drive.mount('/content/drive')

def chebyshev_eeg_filter(data, fs=250, lowcut=8, highcut=30, order=4, rp=0.5):
    """Correct Chebyshev Type I for EEG motor imagery"""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = cheby1(order, rp, [low, high], btype='band')
    return filtfilt(b, a, data, axis=-1)

def process_eeg_folder(input_folder='', output_folder_parent='Subject1_Chebyshev_Filtered'):
    """
    ✅ CORRECTED: Finds ALL Excel files recursively + processes them
    """
    # Your paths
    base_path = Path('/content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1')
    input_folder = base_path / input_folder  # '' = current folder

    # Output to SEPARATE location
    output_base = Path('/content/drive/MyDrive/Processed_EEG_Data')
    output_folder = output_base / output_folder_parent

    # Create output folder
    output_folder.mkdir(parents=True, exist_ok=True)

    # ✅ FIXED: Search RECURSIVELY for ALL Excel files
    print("🔍 Scanning for Excel files...")
    excel_files = []
    for ext in ['*.xlsx', '*.xls']:
        excel_files.extend(list(input_folder.rglob(ext)))
        excel_files.extend(list(base_path.rglob(ext)))  # Also check parent

    print(f"✅ Found {len(excel_files)} Excel files:")
    for f in excel_files[:5]:  # Show first 5
        print(f"   📄 {f}")
    if len(excel_files) > 5:
        print(f"   ... and {len(excel_files)-5} more")

    if len(excel_files) == 0:
        print("❌ NO EXCEL FILES FOUND! Check your folder structure.")
        print("📁 Available folders/files:")
        for item in base_path.iterdir():
            print(f"   {'📁' if item.is_dir() else '📄'} {item.name}")
        return

    print(f"\n💾 Output folder: {output_folder}")

    # Process each file
    for i, file_path in enumerate(excel_files, 1):
        print(f"\n[{i}/{len(excel_files)}] Processing: {file_path.name}")

        try:
            # Read Excel
            df = pd.read_excel(file_path)
            data_raw = df.values

            print(f"   Shape: {data_raw.shape}")
            print(f"   Raw range: {data_raw.min():.2f} to {data_raw.max():.2f} μV")

            # Fix orientation: channels x timepoints
            if data_raw.shape[0] > data_raw.shape[1]:
                data_raw = data_raw.T

            # Chebyshev filtering
            data_cheby = chebyshev_eeg_filter(data_raw, fs=250)

            # Baseline correction
            if data_cheby.shape[1] > 250:
                baseline = data_cheby[:, :250].mean(axis=-1, keepdims=True)
                data_clean = data_cheby - baseline
            else:
                data_clean = data_cheby

            print(f"   Clean range: {data_clean.min():.2f} to {data_clean.max():.2f} μV")

            # Save with original name prefix
            output_filename = f"Cheby_{file_path.stem}.xlsx"
            output_file = output_folder / output_filename
            df_clean = pd.DataFrame(data_clean.T)
            df_clean.to_excel(output_file, index=False)

            print(f"   ✅ Saved: {output_filename}")

        except Exception as e:
            print(f"   ❌ ERROR: {str(e)[:100]}")
            continue

    print(f"\n🎉 BATCH COMPLETE! {len(excel_files)} files processed")
    print(f"📂 All files: {output_folder}")

# 🚀 RUN THIS - Processes ALL Excel files in Subject 1 folder & subfolders
process_eeg_folder('', 'Subject1_Chebyshev_Filtered')


🔍 Scanning for Excel files...
✅ Found 24 Excel files:
   📄 /content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1/ARROW_Forward.xlsx
   📄 /content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1/LETTER_Backward.xlsx
   📄 /content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1/LETTER_Forward.xlsx
   📄 /content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1/WORD_Forward.xlsx
   📄 /content/drive/MyDrive/Human Computer Interface (HCI)/Original Dataset/Subject 1/LETTER_Left.xlsx
   ... and 19 more

💾 Output folder: /content/drive/MyDrive/Processed_EEG_Data/Subject1_Chebyshev_Filtered

[1/24] Processing: ARROW_Forward.xlsx
   Shape: (10240, 20)
   Raw range: -5187.00 to 2045.00 μV
   Clean range: -7.34 to 6.72 μV
   ✅ Saved: Cheby_ARROW_Forward.xlsx

[2/24] Processing: LETTER_Backward.xlsx
   Shape: (10240, 20)
   Raw range: -5186.00 to 2046.00 μV
   Clean range: -7.43 to 6.90 μV
   ✅ Saved: C